In [29]:
import pandas as pd
from transformers import LukeTokenizer, LukeForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup, AdamW
from tqdm import tqdm 

file_path = './dataset.csv'
dataset = pd.read_csv(file_path)

data = dataset.sample(frac=0.01, random_state=42)
data.reset_index(drop=True)
data = data.to_dict(orient="records")


# label2id = {"O": 0, "B-MENTAL_HEALTH_CONCERN": 1, "I-MENTAL_HEALTH_CONCERN": 2}

In [30]:

tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base")
model = LukeForTokenClassification.from_pretrained("studio-ousia/luke-base", num_labels=2)

from transformers import LukeTokenizer

label2id = {"O": 0, "CONCERN": 1}

def label_concern_phrases(user_input, extracted_concern, tokenizer):
    # Tokenize with padding enabled
    tokens = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True )
    labels = [label2id["O"]] * len(tokens["input_ids"][0])

    # Find positions of the extracted concern phrase within the input text
    start_idx = user_input.find(extracted_concern)
    if start_idx != -1:
        end_idx = start_idx + len(extracted_concern)
        concern_tokens = tokenizer(extracted_concern, add_special_tokens=False, return_tensors="pt")["input_ids"][0]

        # Mark tokens in extracted concern as "CONCERN"
        for i, token_id in enumerate(tokens["input_ids"][0]):
            if token_id in concern_tokens:
                labels[i] = label2id["CONCERN"]

    # Pad labels to match input length after tokenization
    labels += [label2id["O"]] * (tokens["input_ids"].shape[1] - len(labels))
    return tokens, labels


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of LukeForTokenClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from torch.utils.data import Dataset, DataLoader

class ConcernDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            user_input = self.data[idx]["User Input"]
            extracted_concern = self.data[idx]["Extracted Concern"]
            tokens, labels = label_concern_phrases(user_input, extracted_concern, self.tokenizer)

            # Convert tokens and labels to tensors
            return {
                "input_ids": tokens["input_ids"].squeeze(0),
                "attention_mask": tokens["attention_mask"].squeeze(0),
                "labels": torch.tensor(labels, dtype=torch.long)
            }
        except KeyError as e:
            print(f"KeyError at index {idx}: {e}")
            raise



In [32]:
dataset = ConcernDataset(data, tokenizer)
from transformers import DataCollatorForTokenClassification

# Initialize the data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Update DataLoader to use the collate function
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=data_collator)


In [33]:
import torch.nn as nn
optimizer = AdamW(model.parameters(), lr=3e-5)
loss_fn = nn.CrossEntropyLoss()
epochs = 5

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [34]:
# for batch in tqdm(train_dataloader):
#     input_ids = batch['input_ids']
#     attention_mask = batch['attention_mask']
#     labels = batch['labels']
    
#     # Print shapes
#     print(f"input_ids shape: {input_ids.shape}")
#     print(f"attention_mask shape: {attention_mask.shape}")
#     print(f"labels shape: {labels.shape}")
#     break

In [39]:
def train(model, train_dataloader, optimizer, epochs=3):
    model.train()
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_loss = 0
        
        for batch in tqdm(train_dataloader):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            print(labels.shape)
            # Zero gradients before each step
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            # print(outputs.logits.shape)
            # loss = outputs.loss
            logits = outputs.logits 
            loss = loss_fn(logits.view(-1,2), labels.view(-1))
            epoch_loss += loss.item()
            
            # Backward pass and optimization step
            loss.backward()
            optimizer.step()
            # scheduler.step()
        
        avg_loss = epoch_loss / len(train_dataloader)
        print(f"Average loss for Epoch {epoch + 1}: {avg_loss:.4f}")

# Start training
train(model, train_dataloader, optimizer)


Epoch 1/3


  8%|▊         | 1/13 [00:00<00:01,  8.71it/s]

torch.Size([8, 19])


 15%|█▌        | 2/13 [00:00<00:02,  3.90it/s]

torch.Size([8, 18])
torch.Size([8, 18])


...

torch.Size([8, 17])


 92%|█████████▏| 12/13 [00:00<00:00, 20.76it/s]

torch.Size([8, 19])
torch.Size([8, 19])
torch.Size([4, 19])


100%|██████████| 13/13 [00:00<00:00, 16.48it/s]

Average loss for Epoch 3: 0.0028


In [36]:
from transformers import LukeForTokenClassification, LukeTokenizer

# Define paths for saving
save_directory = "./saved_model_luke"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./saved_model_luke/tokenizer_config.json',
 './saved_model_luke/special_tokens_map.json',
 './saved_model_luke/vocab.json',
 './saved_model_luke/merges.txt',
 './saved_model_luke/entity_vocab.json',
 './saved_model_luke/added_tokens.json')

In [42]:
model = LukeForTokenClassification.from_pretrained(save_directory)
tokenizer = LukeTokenizer.from_pretrained(save_directory)


model.eval()

def predict_concerns(text, tokenizer, model, label2id):
    # Tokenize the input text with padding and truncation
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    input_ids = tokens["input_ids"]
    attention_mask = tokens["attention_mask"]

    # Perform prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Get the predicted label IDs
    predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()

    # Convert label IDs to label names
    id2label = {v: k for k, v in label2id.items()}
    predicted_labels = [id2label[label_id] for label_id in predictions]

    # Convert tokens to readable text
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze())

    # Only display tokens labeled as concerns
    results = []
    for token, label in zip(tokens, predicted_labels):
        if label != "O":  # Show only tokens with 'CONCERN' labels
            results.append((token, label))
            print(f"{token}: {label}")

    return results


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
text = "I am not eating properly, and it’s sucidal"
print(predict_concerns(text, tokenizer,model,label2id))

Ġeating: CONCERN
Ġproperly: CONCERN
idal: CONCERN
[('Ġeating', 'CONCERN'), ('Ġproperly', 'CONCERN'), ('idal', 'CONCERN')]


In [ ]:
def predict_concerns_dataloader(test_dataloader, tokenizer, model, label2id):
    model.eval()  # Set the model to evaluation mode
    all_results = []  # To store results for all batches

    # Disable gradient calculation for inference
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            # Move batch to device
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']

            # Perform prediction
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Get the predicted label IDs
            predictions = torch.argmax(outputs.logits, dim=-1)

            # Convert label IDs to label names
            id2label = {v: k for k, v in label2id.items()}
            predicted_labels = [id2label[label_id.item()] for label_id in predictions.squeeze()]

            # Convert tokens to readable text
            tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze())

            # Collect results for the current batch
            for token, label in zip(tokens, predicted_labels):
                if label != "O":  # Show only tokens with 'CONCERN' labels
                    all_results.append((token, label))
                    print(f"{token}: {label}")

    return all_results

# Example usage
all_concerns = predict_concerns_dataloader(test_dataloader, tokenizer, model, label2id)
